# Assignment 3

- Go back to Lab assignment 2 and improve the LR result based on the extensions of LR
- Classify the weakest link based on the 38 predictors of a movement using a few classification variants(different models or model parameterizations)
- Select an accuracy metric
- Test and iteratively improve the accuracy of the variants

## Imports

In [ ]:
import sys

sys.path.append("../../scripts")

import dagshub
import mlflow
import ml_utils as MLUtils

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import ExtraTreesClassifier



## Setup

In [ ]:
# Setup dagshub and MLFlow

dagshub.init(repo_owner="SamuelFredricBerg", repo_name="4dt907", mlflow=True)
utils = MLUtils.MLUtils("Project_Model_A3_V2")

# Configs
config = {
    "data_split_seed": 42,
    "test_size": 0.2,
    "n_folds": 10,
    "shuffle": True,
    "variant": "A3-Configurations",
    
    "learning_rate": 0.1,
    "n_estimators": 100,
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf": 5,
    "class_weight": "balanced",

    # "penalty": "l2", 
    # "C": 3.0,             # Strong regularization for stability
    # "solver": "lbfgs",    # Good for multinomial problems
    # "max_iter": 5000
}

WeakLink_data_path = "../../data/AimoScore_WeakLink_big_scores_A3.csv"
Scores_data_path = "../../data/scores_and_weak_links_A3.csv"

dfw = pd.read_csv(WeakLink_data_path)
dfs = pd.read_csv(Scores_data_path)


df_final = pd.merge(dfw, dfs, on="ID")


weak_link_columns = [
    'ForwardHead', 'LeftArmFallForward', 'RightArmFallForward',
    'LeftShoulderElevation', 'RightShoulderElevation', 'ExcessiveForwardLean',
    'LeftAsymmetricalWeightShift', 'RightAsymmetricalWeightShift',
    'LeftKneeMovesInward', 'RightKneeMovesInward', 'LeftKneeMovesOutward',
    'RightKneeMovesOutward', 'LeftHeelRises', 'RightHeelRises'
]

df_final['WeakestLink'] = df_final[weak_link_columns].idxmax(axis=1)


kf = KFold(
    n_splits=config["n_folds"],
    shuffle=config["shuffle"],
    random_state=config["data_split_seed"],
)

# Preprocessing

In [ ]:
def preprocessing(df):
    df_cleaned = df
    return df_cleaned

# Plots

In [ ]:
def confuse_matrix(X, y, model):
    # 1. Train on a single split to see real predictions
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    all_labels = weak_link_columns 

    fig, ax = plt.subplots(figsize=(12, 10)) 
    ConfusionMatrixDisplay.from_predictions(
        y_test, 
        y_pred, 
        ax=ax, 
        display_labels=all_labels, # Force these labels on the axes
        labels=all_labels,         # Ensure it looks for all 14
        cmap='Blues', 
        xticks_rotation=90
    )
    plt.title("Weakest Link Prediction: 14-Class Matrix")
    plt.tight_layout() # Prevents label cutoff
    plt.savefig("matrix.png")
    plt.show()

In [ ]:
def crossval_boxplots(f1_raw, acc_raw, prec_raw, rec_raw):
    # 1. Create a dictionary of the raw arrays
    raw_data = {
        'F1 Score': f1_raw,
        'Accuracy': acc_raw,
        'Precision': prec_raw,
        'Recall': rec_raw
    }
    
    # 2. Convert to 'Long Format' (This is why your plot was likely clamped)
    df_plot = pd.DataFrame(raw_data).melt(var_name='Metric', value_name='Score')
    
    # 3. Plotting
    plt.figure(figsize=(10, 6))
    
    # Setting the style to whitegrid helps readability
    sns.set_style("whitegrid")
    
    # Use 'Metric' for x and 'Score' for y
    ax = sns.boxplot(x='Metric', y='Score', data=df_plot, palette="Set3", width=0.4)
    sns.swarmplot(x='Metric', y='Score', data=df_plot, color=".25", size=7)

    # 4. Smart Scaling: Instead of 0 to 1, let's zoom in on the data 
    # but keep a little padding so we can see the whiskers
    ymin = df_plot['Score'].min() - 0.05
    ymax = df_plot['Score'].max() + 0.05
    plt.ylim(max(0, ymin), min(1, ymax)) # Ensure we don't go out of 0-1 range
    
    plt.title(f'Model Stability: {config["variant"]}\n(Zoomed in to show variance)')
    plt.ylabel('Score Value')
    
    plt.savefig("boxplot.png")
    plt.show()

## Main Code

In [ ]:

with mlflow.start_run(run_name="A3-Training-RFC") as run:
    mlflow.log_params(config)


    y = df_final['WeakestLink']
    cheating_cols = weak_link_columns + [
    'WeakestLink', 'ID', 'Date', 'SCORE', 'AimoScore', 
    'EstimatedScore', 'label score'
    ]

    X = df_final.drop(columns=[c for c in cheating_cols if c in df_final.columns], errors='ignore')

    X = X.select_dtypes(include=[np.number])

    

    model = RandomForestClassifier(
        n_estimators=config["n_estimators"],
        max_depth=config["max_depth"],
        min_samples_split=config["min_samples_split"],
        random_state=config["data_split_seed"],
        class_weight=config["class_weight"],
        min_samples_leaf=config["min_samples_leaf"],
    )
    #model = GradientBoostingClassifier(
    #    n_estimators=config["n_estimators"],
    #    learning_rate=config["learning_rate"], 
    #    max_depth=config["max_depth"],
    #    random_state=config["data_split_seed"],
    #    min_samples_split=config["min_samples_split"],
    #    min_samples_leaf=config["min_samples_leaf"]
    #)
    # model = make_pipeline(
    # StandardScaler(), 
    # LogisticRegression(
    #     penalty=config["penalty"],
    #     C=config["C"],
    #     solver=config["solver"],
    #     max_iter=config["max_iter"],
    #     multi_class='multinomial',
    #     class_weight='balanced',
    #     random_state=config["data_split_seed"]
    #     )
    # )


    # model = ExtraTreesClassifier(
    #     n_estimators=config["n_estimators"],
    #     max_depth=config["max_depth"],
    #     min_samples_leaf=config["min_samples_leaf"],
    #     random_state=config["data_split_seed"],
    #     class_weight=config["class_weight"],
    #     n_jobs=-1
    # )

    print("Model Done")

    model.fit(X, y)

    print("Fitting done")

    confuse_matrix(X, y, model)


    f1_scores = cross_val_score(model, X, y, cv=kf, scoring='f1_weighted')
    accuracy = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    precision = cross_val_score(model, X, y, cv=kf, scoring='precision_weighted')
    recall = cross_val_score(model, X, y, cv=kf, scoring='recall_weighted')

    results = {
        "F1_Mean": f1_scores.mean(),
        "F1_Std": f1_scores.std(),
        "Accuracy_Mean": accuracy.mean(),
        "Precision_Mean": precision.mean(),
        "Recall_Mean": recall.mean()
    }
    print(results)

    crossval_boxplots(f1_scores,accuracy,precision,recall)

    mlflow.log_metrics(results)

    mlflow.log_artifact("boxplot.png")
    mlflow.log_artifact("matrix.png")

    if utils.auto_check_challenger(run.info.run_id, metric_name="F1_Mean"):
        mlflow.sklearn.log_model(model, "model", registered_model_name="Project_Model_A3_V2")
        latest_v = utils.client.get_latest_versions("Project_Model_A3_V2")[0].version
        utils.client.set_registered_model_alias("Project_Model_A3_V2", "dev", latest_v)
        print("New model beat current @dev uploading to DagsHub")
    else:
        print("Did not beat current @dev, model not uploaded to Dagshub")